In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Canny&Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_f1 = np.load('../trainImage_canny.npy')/255.0
        self.test_x_f1 = np.load('../testImage_canny.npy')/255.0
        self.train_x_f2 = np.load('../trainImage_pyramid_L.npy')/255.0
        self.test_x_f2 = np.load('../testImage_pyramid_L.npy')/255.0
        self.train_x_f3 = np.load('../trainImage_pyramid_G.npy')/255.0
        self.test_x_f3 = np.load('../testImage_pyramid_G.npy')/255.0

        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_f1, x_f2, x_f3, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_f1",x_f1.get_shape()) 
            print("CNN:x_f2",x_f2.get_shape())  
            print("CNN:x_f3",x_f3.get_shape())  
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x_f1,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_f1,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())
            
            #输入x_f2,卷积核为3*3 输出维度为32
            net1_3 = tf.layers.conv2d(inputs = x_f2,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_3'               # 命名用于获取变量
                                    )
            print("CNN:",net1_3.get_shape())
            
            #输入x_f3,卷积核为3*3 输出维度为32
            net1_4 = tf.layers.conv2d(inputs = x_f3,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_4'               # 命名用于获取变量
                                    )
            print("CNN:",net1_4.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.concat([net, net1_3], 3)
            net = tf.concat([net, net1_4], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_f1 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f1.shape[1], 
                                                     self.train_x_f1.shape[2], self.train_x_f1.shape[3]],
                                  name='x_f1')
        self.x_f2 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f2.shape[1], 
                                                     self.train_x_f2.shape[2], self.train_x_f2.shape[3]],
                                  name='x_f2')
        self.x_f3 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f3.shape[1], 
                                             self.train_x_f3.shape[2], self.train_x_f3.shape[3]],
                          name='x_f3')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x,  self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_f1 = self.train_x_f1[shuffle_idxs]
            shuffled_set_f2 = self.train_x_f2[shuffle_idxs]
            shuffled_set_f3 = self.train_x_f3[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f1 = shuffled_set_f1[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f2 = shuffled_set_f2[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f3 = shuffled_set_f3[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_f1: batch_x_f1,
                                                          self.x_f2: batch_x_f2,
                                                          self.x_f3: batch_x_f3,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_f1: batch_x_f1_test,
                                                                    self.x_f2: batch_x_f2_test,
                                                                    self.x_f3: batch_x_f3_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.x_f3: batch_x_f3_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [ ]:
dataset = '4_Flowers_1s'
epoch = 80
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 30.02, total 80 epochs ptime: 2459.25
#  [*] Training finished!
#  [*] Best Epoch:  6 , Accuracy:  0.8199999928474426
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-6
#  [*] Finished testing Best Epoch: 6 , accuracy:  0.8199999928474426 !

CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [

Epoch: [ 2] [  31/  60] time: 55.4448, loss: 0.04752878
Epoch: [ 2] [  32/  60] time: 55.9063, loss: 0.03357669
Epoch: [ 2] [  33/  60] time: 56.3653, loss: 0.07339180
Epoch: [ 2] [  34/  60] time: 56.8301, loss: 0.03253435
Epoch: [ 2] [  35/  60] time: 57.2849, loss: 0.10580515
Epoch: [ 2] [  36/  60] time: 57.7483, loss: 0.02205219
Epoch: [ 2] [  37/  60] time: 58.2059, loss: 0.03020087
Epoch: [ 2] [  38/  60] time: 58.6664, loss: 0.02754891
Epoch: [ 2] [  39/  60] time: 59.1328, loss: 0.03808596
Epoch: [ 2] [  40/  60] time: 59.5969, loss: 0.05260148
Epoch: [ 2] [  41/  60] time: 60.0476, loss: 0.07932588
Epoch: [ 2] [  42/  60] time: 60.5100, loss: 0.08025809
Epoch: [ 2] [  43/  60] time: 60.9640, loss: 0.02345762
Epoch: [ 2] [  44/  60] time: 61.4273, loss: 0.05088423
Epoch: [ 2] [  45/  60] time: 61.8998, loss: 0.02185287
Epoch: [ 2] [  46/  60] time: 62.3609, loss: 0.05215775
Epoch: [ 2] [  47/  60] time: 62.8268, loss: 0.02252727
Epoch: [ 2] [  48/  60] time: 63.2839, loss: 0.0

Epoch: [ 4] [  54/  60] time: 129.7614, loss: 0.03567290
Epoch: [ 4] [  55/  60] time: 130.2226, loss: 0.00918499
Epoch: [ 4] [  56/  60] time: 130.6989, loss: 0.03720368
Epoch: [ 4] [  57/  60] time: 131.1612, loss: 0.01753309
Epoch: [ 4] [  58/  60] time: 131.6213, loss: 0.00589909
Epoch: [ 4] [  59/  60] time: 132.0879, loss: 0.00397731
[4/80] - ptime: 30.6089 loss: 0.03506399 acc: 0.23000 lr: 0.00100000
Epoch: [ 5] [   0/  60] time: 136.0800, loss: 0.01711849
Epoch: [ 5] [   1/  60] time: 136.5403, loss: 0.01912150
Epoch: [ 5] [   2/  60] time: 137.0074, loss: 0.04076777
Epoch: [ 5] [   3/  60] time: 137.4806, loss: 0.00619855
Epoch: [ 5] [   4/  60] time: 137.9452, loss: 0.00383411
Epoch: [ 5] [   5/  60] time: 138.4229, loss: 0.01560826
Epoch: [ 5] [   6/  60] time: 138.8821, loss: 0.00881582
Epoch: [ 5] [   7/  60] time: 139.3461, loss: 0.01262252
Epoch: [ 5] [   8/  60] time: 139.8071, loss: 0.02209455
Epoch: [ 5] [   9/  60] time: 140.2654, loss: 0.00409410
Epoch: [ 5] [  10/ 

Epoch: [ 7] [  15/  60] time: 206.0318, loss: 0.01038907
Epoch: [ 7] [  16/  60] time: 206.5003, loss: 0.00957552
Epoch: [ 7] [  17/  60] time: 206.9589, loss: 0.00405834
Epoch: [ 7] [  18/  60] time: 207.4301, loss: 0.00447985
Epoch: [ 7] [  19/  60] time: 207.8879, loss: 0.00450380
Epoch: [ 7] [  20/  60] time: 208.3731, loss: 0.03557557
Epoch: [ 7] [  21/  60] time: 208.8352, loss: 0.04570019
Epoch: [ 7] [  22/  60] time: 209.3010, loss: 0.02358934
Epoch: [ 7] [  23/  60] time: 209.7648, loss: 0.01584306
Epoch: [ 7] [  24/  60] time: 210.2346, loss: 0.01886275
Epoch: [ 7] [  25/  60] time: 210.6945, loss: 0.01630181
Epoch: [ 7] [  26/  60] time: 211.1624, loss: 0.00141925
Epoch: [ 7] [  27/  60] time: 211.6232, loss: 0.00262656
Epoch: [ 7] [  28/  60] time: 212.0787, loss: 0.01553013
Epoch: [ 7] [  29/  60] time: 212.5411, loss: 0.02184606
Epoch: [ 7] [  30/  60] time: 213.0112, loss: 0.03161281
Epoch: [ 7] [  31/  60] time: 213.4713, loss: 0.01690108
Epoch: [ 7] [  32/  60] time: 2

Epoch: [ 9] [  37/  60] time: 279.1591, loss: 0.01183810
Epoch: [ 9] [  38/  60] time: 279.6484, loss: 0.07133423
Epoch: [ 9] [  39/  60] time: 280.1124, loss: 0.00123322
Epoch: [ 9] [  40/  60] time: 280.5682, loss: 0.06644503
Epoch: [ 9] [  41/  60] time: 281.0318, loss: 0.01928633
Epoch: [ 9] [  42/  60] time: 281.4868, loss: 0.00923112
Epoch: [ 9] [  43/  60] time: 281.9427, loss: 0.00404619
Epoch: [ 9] [  44/  60] time: 282.4092, loss: 0.02785806
Epoch: [ 9] [  45/  60] time: 282.8623, loss: 0.01100358
Epoch: [ 9] [  46/  60] time: 283.3408, loss: 0.06578665
Epoch: [ 9] [  47/  60] time: 283.8036, loss: 0.00506154
Epoch: [ 9] [  48/  60] time: 284.2661, loss: 0.00760707
Epoch: [ 9] [  49/  60] time: 284.7349, loss: 0.03094110
Epoch: [ 9] [  50/  60] time: 285.1961, loss: 0.04060678
Epoch: [ 9] [  51/  60] time: 285.6531, loss: 0.00098354
Epoch: [ 9] [  52/  60] time: 286.1225, loss: 0.02279900
Epoch: [ 9] [  53/  60] time: 286.5838, loss: 0.00028688
Epoch: [ 9] [  54/  60] time: 2

...

Epoch: [70] [  11/  60] time: 2140.2741, loss: 0.00000061
Epoch: [70] [  12/  60] time: 2140.7267, loss: 0.00000169
Epoch: [70] [  13/  60] time: 2141.1703, loss: 0.00000011
Epoch: [70] [  14/  60] time: 2141.6187, loss: 0.00000026
Epoch: [70] [  15/  60] time: 2142.0670, loss: 0.00000200
Epoch: [70] [  16/  60] time: 2142.5077, loss: 0.00000144
Epoch: [70] [  17/  60] time: 2142.9555, loss: 0.00000116
Epoch: [70] [  18/  60] time: 2143.4152, loss: 0.00000450
Epoch: [70] [  19/  60] time: 2143.8621, loss: 0.00000091
Epoch: [70] [  20/  60] time: 2144.3156, loss: 0.00000364
Epoch: [70] [  21/  60] time: 2144.7676, loss: 0.00000032
Epoch: [70] [  22/  60] time: 2145.2150, loss: 0.00000412
Epoch: [70] [  23/  60] time: 2145.6625, loss: 0.00000277
Epoch: [70] [  24/  60] time: 2146.1098, loss: 0.00000148
Epoch: [70] [  25/  60] time: 2146.5620, loss: 0.00000074
Epoch: [70] [  26/  60] time: 2147.0107, loss: 0.00000211
Epoch: [70] [  27/  60] time: 2147.4713, loss: 0.00000012
Epoch: [70] [ 

Epoch: [72] [  30/  60] time: 2208.9138, loss: 0.00000254
Epoch: [72] [  31/  60] time: 2209.3556, loss: 0.00000117
Epoch: [72] [  32/  60] time: 2209.8039, loss: 0.00000006
Epoch: [72] [  33/  60] time: 2210.2593, loss: 0.00000303
Epoch: [72] [  34/  60] time: 2210.7052, loss: 0.00000009
Epoch: [72] [  35/  60] time: 2211.1570, loss: 0.00000067
Epoch: [72] [  36/  60] time: 2211.6073, loss: 0.00000440
Epoch: [72] [  37/  60] time: 2212.0581, loss: 0.00000047
Epoch: [72] [  38/  60] time: 2212.5050, loss: 0.00000784
Epoch: [72] [  39/  60] time: 2212.9476, loss: 0.00000051
Epoch: [72] [  40/  60] time: 2213.4021, loss: 0.00000555
Epoch: [72] [  41/  60] time: 2213.8645, loss: 0.00000014
Epoch: [72] [  42/  60] time: 2214.3174, loss: 0.00001104
Epoch: [72] [  43/  60] time: 2214.7652, loss: 0.00000017
Epoch: [72] [  44/  60] time: 2215.2158, loss: 0.00000003
Epoch: [72] [  45/  60] time: 2215.6701, loss: 0.00000251
Epoch: [72] [  46/  60] time: 2216.1195, loss: 0.00000011
Epoch: [72] [ 

Epoch: [74] [  49/  60] time: 2277.6117, loss: 0.00000039
Epoch: [74] [  50/  60] time: 2278.0653, loss: 0.00000140
Epoch: [74] [  51/  60] time: 2278.5128, loss: 0.00003623
Epoch: [74] [  52/  60] time: 2278.9659, loss: 0.00000082
Epoch: [74] [  53/  60] time: 2279.4168, loss: 0.00000005
Epoch: [74] [  54/  60] time: 2279.8672, loss: 0.00000035
Epoch: [74] [  55/  60] time: 2280.3133, loss: 0.00000399
Epoch: [74] [  56/  60] time: 2280.7574, loss: 0.00000248
Epoch: [74] [  57/  60] time: 2281.2053, loss: 0.00000565
Epoch: [74] [  58/  60] time: 2281.6544, loss: 0.00000147
Epoch: [74] [  59/  60] time: 2282.1055, loss: 0.00000009
[74/80] - ptime: 29.5047 loss: 0.00000599 acc: 0.73000 lr: 0.00047830
Epoch: [75] [   0/  60] time: 2285.7624, loss: 0.00000025
Epoch: [75] [   1/  60] time: 2286.2072, loss: 0.00001122
Epoch: [75] [   2/  60] time: 2286.6495, loss: 0.00000009
Epoch: [75] [   3/  60] time: 2287.1109, loss: 0.00000005
Epoch: [75] [   4/  60] time: 2287.5588, loss: 0.00000070
Ep

Epoch: [77] [   7/  60] time: 2348.0618, loss: 0.00000063
Epoch: [77] [   8/  60] time: 2348.5026, loss: 0.00000040
Epoch: [77] [   9/  60] time: 2348.9506, loss: 0.00000313
Epoch: [77] [  10/  60] time: 2349.3943, loss: 0.00000696
Epoch: [77] [  11/  60] time: 2349.8381, loss: 0.00000049
Epoch: [77] [  12/  60] time: 2350.2827, loss: 0.00001786
Epoch: [77] [  13/  60] time: 2350.7334, loss: 0.00000031
Epoch: [77] [  14/  60] time: 2351.1760, loss: 0.00000064
Epoch: [77] [  15/  60] time: 2351.6227, loss: 0.00000014
Epoch: [77] [  16/  60] time: 2352.0685, loss: 0.00000553
Epoch: [77] [  17/  60] time: 2352.5146, loss: 0.00000008
Epoch: [77] [  18/  60] time: 2352.9609, loss: 0.00000698
Epoch: [77] [  19/  60] time: 2353.4085, loss: 0.00000523
Epoch: [77] [  20/  60] time: 2353.8530, loss: 0.00000416
Epoch: [77] [  21/  60] time: 2354.2935, loss: 0.00000014
Epoch: [77] [  22/  60] time: 2354.7358, loss: 0.00000012
Epoch: [77] [  23/  60] time: 2355.1800, loss: 0.00000137
Epoch: [77] [ 

Epoch: [79] [  26/  60] time: 2414.8701, loss: 0.00000756
Epoch: [79] [  27/  60] time: 2415.3153, loss: 0.00000452
Epoch: [79] [  28/  60] time: 2415.7540, loss: 0.00000109
Epoch: [79] [  29/  60] time: 2416.1984, loss: 0.00000037
Epoch: [79] [  30/  60] time: 2416.6393, loss: 0.00000004
Epoch: [79] [  31/  60] time: 2417.0805, loss: 0.00000027
Epoch: [79] [  32/  60] time: 2417.5214, loss: 0.00000040
Epoch: [79] [  33/  60] time: 2417.9633, loss: 0.00000033
Epoch: [79] [  34/  60] time: 2418.4080, loss: 0.00000070
Epoch: [79] [  35/  60] time: 2418.8490, loss: 0.00001109
Epoch: [79] [  36/  60] time: 2419.2925, loss: 0.00000155
Epoch: [79] [  37/  60] time: 2419.7387, loss: 0.00000013
Epoch: [79] [  38/  60] time: 2420.1818, loss: 0.00000005
Epoch: [79] [  39/  60] time: 2420.6288, loss: 0.00000368
Epoch: [79] [  40/  60] time: 2421.0734, loss: 0.00000047
Epoch: [79] [  41/  60] time: 2421.5218, loss: 0.00000115
Epoch: [79] [  42/  60] time: 2421.9588, loss: 0.00000067
Epoch: [79] [ 

In [ ]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放